In [128]:
import numpy as np
from hashlib import sha256, sha1
from numpy.random import randint
from math import ceil

In [112]:
# Parameters
NONCE_LOW = 0
NONCE_HIGH = 100

class Blockchain():
    """A collection of Block objects linked together through the hash
    of the previous block in the chain.
    """
    
    def __init__(self, head):
        self.head = head
        
    def addBlock(self, block):
        block.next = self.head
        block.hash = block.goalHash()
        self.head = block
    
    def size(self):
        temp = self.head
        nextBlock = temp.nextBlock
        k = 0
        while nextBlock:
            temp = nextBlock
            nextBlock = nextBlock.nextBlock
            k += 1
        return k
    
    def latestBlock(self):
        return self.head

def packageTransaction(transactionList):
    """Takes in a list of transactions (max of 4 in a block) and
    formats it into a string that can be used to calculate the
    hash of a block.
    """
    formatted = ''
    for transaction in transactionList:
        formatted += str(transaction) + ';'
    return formatted

class Block():
    """An individual block that contains a signed hash (that fulfills
    certain criteria), data in the form of transactions, and the previous
    block's hash. Once a miner finds a suitable signed hash, a block
    is created and the miner is rewarded tokens.
    """    
    def __init__(self, index, previousBlock, data, nonce):
        self.previousBlock = previousBlock
        self.index = index        
        self.hash = None
        
        if previousBlock and data and nonce:
            self.data = data
            self.nonce = nonce
            self.previousBlock = previousBlock
        else:
            self.hash = sha256('0'.encode()).hexdigest()
        
        self.signedBy = None # whoever finds the solution first signs
        
        self.possibleNextBlocks = []
        self.nextBlock = None
    
    def sign(self, miner):
        self.signedBy = miner
        miner.blocksSigned += 1
        miner.reward += 12.5
        
    # This is for this mini example only, not actually what happens.
    def goalHash(self):            
        formattedTx = packageTransaction(self.data)
        # Nonce + Transaction Data + Previous Hash
        goalHash = str(self.nonce) + formattedTx + self.previousBlock.hash
        return makeHash(goalHash)
    
class User():
    """Someone who is making transactions in the network.
    """
    
    def __init__(self, userId):
        self.userId = userId
        self.balance = 100 # Initialize everyone to have some starting balance
        
    def __str__(self):
        return str(self.userId)
    
class Miner():
    """A miner 
    """
    def __init__(self, minerId):
        self.minerId = minerId
        self.reward = 0
        self.blocksSigned = 0
        
    def mine(self, block):
        randNum = randint(NONCE_LOW, NONCE_HIGH)
        guess, k = randint(NONCE_LOW, NONCE_HIGH), 0
        guessHash = ''
        goalHash = block.goalHash()
        while guessHash != goalHash:
            guess = randint(NONCE_LOW, NONCE_HIGH)
            hashStr = str(guess) + packageTransaction(block.data) + block.previousBlock.hash
            guessHash = makeHash(hashStr)
            k += 1 # Incrememnt the number of computations used          
        return k
    
    def validate(self, blockTransaction, ledgerTransaction):
        if len(blockTransaction) != len(ledgerTransaction):
            return False
        for i in range(len(blockTransaction)):
            if blockTransaction[i] != ledgerTransaction[i]:
                return False
        return True
            
    def __str__(self):
        return 'Miner #' + str(self.minerId)

class BadMiner(Miner):
    """A nefarious miner who wants to mutate the chain to create
    fake/false transactions.
    """ 
    def validate(self, blockTransaction, ledgerTransaction):
        return not super(BadMiner, self).validate(blockTransaction, ledgerTransaction)
    
    def __str__(self):
        return 'Bad Miner #' + str(self.minerId) 
        
class Transaction():
    def __init__(self, fromUser, toUser, amount):        
        # user1 -- amount -> user2
        self.fromUser = fromUser
        self.toUser = toUser
        self.amount = amount
        self.valid = (fromUser.balance - amount) > 0
        
    def __str__(self):
        return 'User ' + str(self.fromUser) + ' sends ' + str(self.amount) + ' to User ' + str(self.toUser)
    
    def __eq__(self, other):
        return self.fromUser.userId == other.fromUser.userId and self.toUser.userId == other.toUser.userId and self.amount == other.amount
    
    def __ne__(self, other):
        return not self.__eq__(other)
    
    def make():
        pass
    
def makeHash(s, f=sha256):
    return f(s.encode()).hexdigest()

In [148]:
# Create network of miners
networkSize = 100

# Create test users
numUsers = 4
user0, user1, user2, user3 = [User(i) for i in range(numUsers)]

# Initialize blockchain
blockchain = Blockchain(Block(0, None, None, None))

# Assume blocks can only hold 4 transactions
# Test for 3 blocks
testTransactions = [
    [Transaction(user0, user1, 20), 
     Transaction(user2, user3, 30), 
     Transaction(user3, user1, 3), 
     Transaction(user3, user0, 15)],
    
    [Transaction(user1, user0, 5), 
     Transaction(user3, user0, 2), 
     Transaction(user1, user2, 3), 
     Transaction(user2, user0, 30)],
    
    [Transaction(user3, user0, 60), 
     Transaction(user2, user3, 30), 
     Transaction(user3, user1, 1), 
     Transaction(user0, user3, 12)]
]

fakeTransaction = [
    Transaction(user1, user0, 100), 
    Transaction(user2, user0, 100), 
    Transaction(user3, user0, 50), 
    Transaction(user3, user0, 50)
]

def runSimulation(percentBad=0):
    goodMiners = int(networkSize * (1 - percentBad))
    # Create miner network with x good miners and networkSize - x bad miners
    minerNetwork = [Miner(i) for i in range(goodMiners)] + [BadMiner(i) for i in range(goodMiners, networkSize)]
    
    for testTransaction in testTransactions:
        fakeNonce = randint(NONCE_LOW, NONCE_HIGH)
        realBlock = Block(blockchain.size(), blockchain.latestBlock(), testTransaction, fakeNonce)
        fakeBlock = Block(blockchain.size(), blockchain.latestBlock(), fakeTransaction, fakeNonce)
        
        minerComputations = []
        for miner in minerNetwork:
            newBlock = realBlock
            if isinstance(miner, BadMiner):
                newBlock = fakeBlock

            if newBlock not in blockchain.latestBlock().possibleNextBlocks:
                blockchain.latestBlock().possibleNextBlocks.append(newBlock)
            minerComputations.append(miner.mine(newBlock))
                
        totalComputations = sum(minerComputations)
        # print(minerComputations, totalComputations)
        # Choose the winning miner based on the miner that takes the least number
        # of computations to get a valid hash.
        winningMiner = minerNetwork[np.argmin(minerComputations)]
        # This miner gets to sign off the new block, and is rewarded with tokens.
        newBlock.sign(winningMiner)
            
        for possibleBlock in blockchain.latestBlock().possibleNextBlocks:
            validation = [miner.validate(possibleBlock.data, testTransaction) for miner in minerNetwork]
            percentValidation = np.mean(validation)
            
            blockchain.addBlock(possibleBlock)
            for data in blockchain.latestBlock().data:
                print(data)
            
            if possibleBlock is fakeBlock:
                print("Fake Block Validation: ", percentValidation)
            if possibleBlock is realBlock:
                print("Real Block Validation: ", percentValidation)
                    
            if percentValidation > 0.5:
                if possibleBlock is realBlock:
                    print('Majority rules. Main chain is safe!\n')
                elif possibleBlock is fakeBlock:
                    print('51% attack has occurred. Main chain now contains the fake transaction!')
                


In [149]:
# All miners are good.
runSimulation()

User 0 sends 20 to User 1
User 2 sends 30 to User 3
User 3 sends 3 to User 1
User 3 sends 15 to User 0
Real Block Validation:  1.0
Majority rules. Main chain is safe!

User 1 sends 5 to User 0
User 3 sends 2 to User 0
User 1 sends 3 to User 2
User 2 sends 30 to User 0
Real Block Validation:  1.0
Majority rules. Main chain is safe!

User 3 sends 60 to User 0
User 2 sends 30 to User 3
User 3 sends 1 to User 1
User 0 sends 12 to User 3
Real Block Validation:  1.0
Majority rules. Main chain is safe!



In [150]:
# One bad miner
runSimulation(percentBad=0.01)

User 0 sends 20 to User 1
User 2 sends 30 to User 3
User 3 sends 3 to User 1
User 3 sends 15 to User 0
Real Block Validation:  0.99
Majority rules. Main chain is safe!

User 1 sends 100 to User 0
User 2 sends 100 to User 0
User 3 sends 50 to User 0
User 3 sends 50 to User 0
Fake Block Validation:  0.01
User 1 sends 5 to User 0
User 3 sends 2 to User 0
User 1 sends 3 to User 2
User 2 sends 30 to User 0
Real Block Validation:  0.99
Majority rules. Main chain is safe!

User 1 sends 100 to User 0
User 2 sends 100 to User 0
User 3 sends 50 to User 0
User 3 sends 50 to User 0
Fake Block Validation:  0.01
User 3 sends 60 to User 0
User 2 sends 30 to User 3
User 3 sends 1 to User 1
User 0 sends 12 to User 3
Real Block Validation:  0.99
Majority rules. Main chain is safe!

User 1 sends 100 to User 0
User 2 sends 100 to User 0
User 3 sends 50 to User 0
User 3 sends 50 to User 0
Fake Block Validation:  0.01


In [151]:
# 51% attack
# Create network of miners with one bad guy
runSimulation(percentBad=0.51)
    
print('Oh no! User 0 gets all the crypto! :(')

User 0 sends 20 to User 1
User 2 sends 30 to User 3
User 3 sends 3 to User 1
User 3 sends 15 to User 0
Real Block Validation:  0.49
User 1 sends 100 to User 0
User 2 sends 100 to User 0
User 3 sends 50 to User 0
User 3 sends 50 to User 0
Fake Block Validation:  0.51
51% attack has occurred. Main chain now contains the fake transaction!
User 1 sends 5 to User 0
User 3 sends 2 to User 0
User 1 sends 3 to User 2
User 2 sends 30 to User 0
Real Block Validation:  0.49
User 1 sends 100 to User 0
User 2 sends 100 to User 0
User 3 sends 50 to User 0
User 3 sends 50 to User 0
Fake Block Validation:  0.51
51% attack has occurred. Main chain now contains the fake transaction!
User 3 sends 60 to User 0
User 2 sends 30 to User 3
User 3 sends 1 to User 1
User 0 sends 12 to User 3
Real Block Validation:  0.49
User 1 sends 100 to User 0
User 2 sends 100 to User 0
User 3 sends 50 to User 0
User 3 sends 50 to User 0
Fake Block Validation:  0.51
51% attack has occurred. Main chain now contains the fake t

TypeError: unhashable type: 'list'